In [ ]:
!uv pip install sagemaker

In [4]:
import sagemaker
import boto3

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\bored\AppData\Local\sagemaker\sagemaker\config.yaml


In [5]:
from sagemaker.session import TrainingInput
from sagemaker.estimator import Estimator
from sagemaker import image_uris
from sagemaker import hyperparameters

In [6]:
region = boto3.Session().region_name

In [7]:
region

'us-east-1'

In [8]:
region = sagemaker.Session().boto_region_name

In [9]:
region

'us-east-1'

In [10]:
import os

In [ ]:
import boto3
import json

# Initialize clients
iam_client = boto3.client('iam')

# Role name
role_name = "SageMakerExecutionRole"

# Trust policy as JSON string
trust_policy = json.dumps({
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": { "Service": "sagemaker.amazonaws.com" },
            "Action": "sts:AssumeRole"
        }
    ]
})

# Create the role
try:
    response = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=trust_policy,
        Description='SageMaker execution role created via boto3'
    )
    print("Role created successfully:", response['Role']['Arn'])

except iam_client.exceptions.EntityAlreadyExistsException:
    print(f"Role {role_name} already exists.")

# Attach managed policy AmazonSageMakerFullAccess
iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonSageMakerFullAccess'
)

iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess'
)

print("Policies attached successfully.")


In [2]:
role_arn=response['Role']['Arn']

In [11]:
role_arn=sagemaker.get_execution_role()

Couldn't call 'get_role' to get Role ARN from role name AgenticPrahalad to get Role path.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 role_arn=sagemaker.get_execution_role()                                                      │
│   2                                                                                              │
│                                                                                                  │
│ c:\Users\bored\Music\SagemekerLearning\sagelearner\lib\site-packages\sagemaker\session.py:8078   │
│ in get_execution_role                                                                            │
│                                                                                                  │
│   8075 │   │   "The current AWS identity is not a role: {}, therefore it cannot be used as a "   │
│   8076 │   │   "SageMaker execution role"                                                        │
│   8077 │   )                                                                                     │
│ ❱ 8078 │   raise ValueError(message.format(arn))                                                 │
│   8079                                                                                           │
│   8080                                                                                           │
│   8081 def generate_default_sagemaker_bucket_name(boto_session):                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: The current AWS identity is not a role: arn:aws:iam::396913733336:user/AgenticPrahalad, therefore it 
cannot be used as a SageMaker execution role

In [12]:
role = role_arn

In [13]:
s3=boto3.client("s3")

In [14]:
s3.list_buckets()

{'ResponseMetadata': {'RequestId': 'NB8BQ8MH8NQS0GKH',
  'HostId': 'XMwyXnANcDUtZTEyAoXo73hyrftcIfFbKtp/EGwrS1+yALEu5rFOzEBfeJV9Hkq6o9r9oiXASGw=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'XMwyXnANcDUtZTEyAoXo73hyrftcIfFbKtp/EGwrS1+yALEu5rFOzEBfeJV9Hkq6o9r9oiXASGw=',
   'x-amz-request-id': 'NB8BQ8MH8NQS0GKH',
   'date': 'Sun, 08 Jun 2025 11:34:59 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'awsagenticbuck1',
   'CreationDate': datetime.datetime(2025, 6, 1, 13, 0, 43, tzinfo=tzutc())},
  {'Name': 'cdk-hnb659fds-assets-396913733336-us-east-1',
   'CreationDate': datetime.datetime(2025, 6, 5, 5, 11, 41, tzinfo=tzutc())},
  {'Name': 'sagemaker-studio-396913733336-3bshcwl60rf',
   'CreationDate': datetime.datetime(2025, 6, 2, 3, 56, 11, tzinfo=tzutc())},
  {'Name': 'sagemaker-studio-396913733336-f8do697lcln',
   'CreationDate': datetime.datetime(2025, 6, 2, 3, 56, 14, tz

In [15]:
bucket="sgmkr-learner"

In [16]:
prefix="iris"

In [17]:
!aws s3 ls {bucket}/{prefix}/

                           PRE batch_transform/
                           PRE data/


In [18]:
!aws s3 ls {bucket}/{prefix}/data/ --recursive

2025-06-07 15:11:26        950 iris/data/iris_test.csv
2025-06-07 15:11:23       1900 iris/data/iris_train.csv


In [19]:
train_file = "data/iris_train.csv"
valid_file = "data/iris_test.csv"

train_file_uri = "s3://{}/{}/{}".format(bucket, prefix, train_file)
valid_file_uri = "s3://{}/{}/{}".format(bucket, prefix, valid_file)
print("train file uri:", train_file_uri)
print("valid file uri:", valid_file_uri)

train file uri: s3://sgmkr-learner/iris/data/iris_train.csv
valid file uri: s3://sgmkr-learner/iris/data/iris_test.csv


In [20]:
train_ip = TrainingInput(train_file_uri, content_type="csv")
print(train_ip)

In [21]:
valid_ip = TrainingInput(valid_file_uri, content_type="csv")
print(valid_ip)

In [22]:
model_op = "s3://{}/{}/{}".format(bucket, prefix, "model")
print(model_op)

s3://sgmkr-learner/iris/model


In [23]:
model_img = sagemaker.image_uris.retrieve("xgboost", region, "latest")
print(model_img)

811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest


In [24]:
base_job_name = "iris-xgboost-"

In [25]:
xgb_model = sagemaker.estimator.Estimator(
    image_uri=model_img,
    role=role_arn,
    base_job_name=base_job_name,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path=model_op,
    sagemaker_session=sagemaker.Session(),
    volume_size=5,
)

In [26]:
xgb_model.set_hyperparameters(
    num_class=3, max_depth=5, num_round=10, objective="multi:softmax",
)

In [29]:
from datetime import datetime

In [34]:
job_name = base_job_name + datetime.today().strftime("%Y-%m-%d-%H-%M-%S")
print(job_name)

iris-xgboost-2025-06-08-17-43-29


In [ ]:
xgb_model.fit({"train": train_ip, "validation": valid_ip}, wait=True, job_name=job_name)

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: iris-xgboost-2025-06-08-17-43-29


2025-06-08 12:13:35 Starting - Starting the training job...
2025-06-08 12:14:00 Starting - Preparing the instances for training...
2025-06-08 12:14:32 Downloading - Downloading input data...
2025-06-08 12:14:57 Downloading - Downloading the training image...
2025-06-08 12:16:03 Training - Training image download completed. Training in progress...Arguments: train
[2025-06-08:12:16:14:INFO] Running standalone xgboost training.
[2025-06-08:12:16:14:INFO] File size need to be processed in the node: 0.0mb. Available memory size in the node: 8555.55mb
[2025-06-08:12:16:14:INFO] Determined delimiter of CSV input is ','
[12:16:14] S3DistributionType set as FullyReplicated
[12:16:14] 100x4 matrix with 400 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-06-08:12:16:14:INFO] Determined delimiter of CSV input is ','
[12:16:14] S3DistributionType set as FullyReplicated
[12:16:14] 50x4 matrix with 200 entries loaded from /opt/ml/input/data/validation?format=c

In [36]:
!aws s3 ls {bucket}/{prefix}/model/

                           PRE iris-xgboost-2025-06-08-17-30-29/
                           PRE iris-xgboost-2025-06-08-17-43-29/


In [37]:
!aws s3 ls {bucket}/{prefix}/model/{job_name}/

                           PRE debug-output/
                           PRE output/
                           PRE profiler-output/


In [38]:
!aws s3 cp s3://{bucket}/{prefix}/model/{job_name}/ ./model/ --recursive


download: s3://sgmkr-learner/iris/model/iris-xgboost-2025-06-08-17-43-29/profiler-output/system/training_job_end.ts to model\profiler-output\system\training_job_end.ts
Completed 0 Bytes/441.9 KiB (0 Bytes/s) with 6 file(s) remaining
download: s3://sgmkr-learner/iris/model/iris-xgboost-2025-06-08-17-43-29/debug-output/training_job_end.ts to model\debug-output\training_job_end.ts
Completed 0 Bytes/441.9 KiB (0 Bytes/s) with 5 file(s) remaining
download: s3://sgmkr-learner/iris/model/iris-xgboost-2025-06-08-17-43-29/profiler-output/framework/training_job_end.ts to model\profiler-output\framework\training_job_end.ts
Completed 0 Bytes/441.9 KiB (0 Bytes/s) with 4 file(s) remaining
Completed 3.4 KiB/441.9 KiB (1.5 KiB/s) with 4 file(s) remaining
download: s3://sgmkr-learner/iris/model/iris-xgboost-2025-06-08-17-43-29/output/model.tar.gz to model\output\model.tar.gz
Completed 3.4 KiB/441.9 KiB (1.5 KiB/s) with 3 file(s) remaining
Completed 104.3 KiB/441.9 KiB (37.2 KiB/s) with 3 file(s) remai

In [1]:
type(xgb_model)

NameError: name 'xgb_model' is not defined